In [293]:
from requests_html import HTMLSession
from scrapy.selector import Selector
import json, fitz, re, time, os
from itertools import repeat
from concurrent.futures import ThreadPoolExecutor

In [60]:
url = "https://mjas.analis.com.my/index.php/list-of-issues-2/"
r = HTMLSession().get(url=url)

In [241]:
Selector(HTMLSession().get(list_of_volumes[1])).css('colspan::text').get()

'Malaysian Journal of Analytical Sciences Vol 27 No 5 2023'

In [272]:
list_of_volumes = Selector(r).css('.elementor-clearfix a').css('a::attr(href)').getall()
pattern = re.compile('[\\\/\?\%\>\<\:,\\n\|]')

def get_data(url):
    # for v in url:
    print(url)
    if "http://https:://" in url:
        url = url.replace('https:://','')
    sess = HTMLSession().get(url)
    sel = Selector(sess)

    journal_series = sel.css('colspan::text').get()

    list_of_pdf_url = sel.xpath("//a[@target='frame_c']/@href").getall()
    for link in list_of_pdf_url:
        download_link = ('https://mjas.analis.com.my/mjas/' + url.split('/')[-2] + '/' + link) #.replace(' ','%')

        filename = journal_series + ' - ' + link.replace('pdf','') + 'pdf'
        # Ensure filename is compatible with OS
        file_name_clean = re.sub(pattern, '', filename)[:250]

        if "html" in file_name_clean:
            pass
        else :
            print(file_name_clean)
            print(download_link)
            with open('pdf/' + file_name_clean, 'wb') as f:
                f.write(HTMLSession().get(download_link).content)
                f.close()


In [266]:
pool = ThreadPoolExecutor(max_workers=3)
pool.map(get_data, [x for x in list_of_volumes])

https://mjas.analis.com.my/mjas/v27_n6/v27_n6.html
https://mjas.analis.com.my/mjas/v27_n5/v27_n5.html
https://mjas.analis.com.my/mjas/v27_n4/v27_n4.html


<generator object Executor.map.<locals>.result_iterator at 0x000002118006DA40>

Malaysian Journal of Analytical Sciences Vol 27 No 6 2023 - Mat Yatim_27_6_1.pdf
Malaysian Journal of Analytical Sciences Vol 27 No 4 2023 - Abd20Hamid_27_4_1.pdf
Malaysian Journal of Analytical Sciences Vol 27 No 5 2023 - Rosdi_27_5_1.pdf
Malaysian Journal of Analytical Sciences Vol 27 No 6 2023 - Pungot_27_6_2.pdf
Malaysian Journal of Analytical Sciences Vol 27 No 5 2023 - Abdul Halim_27_5_2.pdf
Malaysian Journal of Analytical Sciences Vol 27 No 4 2023 - Abdul20Wahab_27_4_2.pdf
Malaysian Journal of Analytical Sciences Vol 27 No 5 2023 - Zolekafeli_27_5_3.pdf
Malaysian Journal of Analytical Sciences Vol 27 No 6 2023 - Keasavan_27_6_3.pdf
Malaysian Journal of Analytical Sciences Vol 27 No 4 2023 - Roslan_27_4_3.pdf
Malaysian Journal of Analytical Sciences Vol 27 No 5 2023 - Sazali_27_5_4.pdf
Malaysian Journal of Analytical Sciences Vol 27 No 6 2023 - Zaharullil_27_6_4.pdf
Malaysian Journal of Analytical Sciences Vol 27 No 4 2023 - Mohd20Nor_27_4_4.pdf
Malaysian Journal of Analytical Sc

In [305]:
def parse_pdf(list_of_filename:list[str]):
    data = []
    for filename in list_of_filename:
        doc = fitz.open(f'pdf/{filename}')
        text = []
        number_of_pages = len(doc)
        for page in range(number_of_pages):
            if page == 0:
                x0 = doc[0].rect.width * 0.05 # kiri ke kanan (yg kiri punya)
                x1 = doc[0].rect.width  # kiri ke kanan (yg kanan punya)
                y0 = doc[0].rect.height * 0.3  # atas ke bawah (yg atas punya)
                y1 = doc[0].rect.height * 0.93   # atas ke bawah (yg bawah punya)

                text_temp = ''.join(doc[page].get_text(clip=fitz.Rect(x0, y0, x1, y1)).replace('\n', ''))
                text.append(text_temp)
            else:
                x0 = doc[0].rect.width * 0.05 # kiri ke kanan (yg kiri punya)
                x1 = doc[0].rect.width  # kiri ke kanan (yg kanan punya)
                y0 = doc[0].rect.height * 0.1  # atas ke bawah (yg atas punya)
                y1 = doc[0].rect.height * 0.93   # atas ke bawah (yg bawah punya)

                text_temp = ''.join(doc[page].get_text(clip=fitz.Rect(x0, y0, x1, y1)).replace('\n', ''))
                text.append(text_temp)

        text = ''.join(text)
        items = json.dumps(
            {
                "filename" : filename,
                "text": text
            }
        )

        data.append(items)

    print(len(data))
    with open(f'data/data.json', 'w') as f:
        f.write(json.dumps(data))

In [306]:
list_of_filename = os.listdir('./pdf')
parse_pdf(list_of_filename)

220


In [307]:
len(list_of_filename)

220